In [6]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import load_model
from err import error_count, calc_diff
from visual import plot
import yfinance as yf


############ Data Preprocessing ############
# Importing the dataset
ds = yf.download('0005.hk',"2017-09-28","2021-09-24")
dataset = ds.iloc[:, :5]
dataset



[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close
Date,,,,,
2017-09-28,76.400002,77.099998,76.050003,76.199997,64.386993
2017-09-29,76.500000,76.900002,76.300003,76.550003,64.682732
2017-10-03,76.800003,77.449997,76.699997,77.449997,65.443222
2017-10-04,77.500000,77.949997,77.500000,77.849998,65.781197
2017-10-06,77.599998,77.800003,77.199997,77.349998,65.358711
...,...,...,...,...,...
2021-09-16,40.599998,40.599998,39.599998,40.000000,40.000000
2021-09-17,39.700001,40.400002,39.099998,40.250000,40.250000
2021-09-20,39.900002,39.950001,38.950001,39.000000,39.000000


In [2]:

X = ds.iloc[:, 2].values
y = ds.iloc[:, 5].values

# Feature Scaling
scaler  = MinMaxScaler(feature_range=(0, 1))
dataset_scaled = scaler.fit_transform(dataset)

X = dataset_scaled[:, 0]
y = dataset_scaled[:, 1]

# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Sizes of train_ds, test_ds
dataset_sz = X.shape[0]
train_sz = X_train.shape[0]
test_sz = X_test.shape[0]

############ Building the ANN ############
# Initialising the ANN
regressor = Sequential()

# Adding the input layer and the first hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu', input_dim = 1))
regressor.add(Dropout(.2))

# Adding the first hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the second hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the third hidden layer and Drop out Regularization
regressor.add(Dense(units = 500, kernel_initializer = 'uniform', activation = 'relu'))
regressor.add(Dropout(.2))

# Adding the output layer
regressor.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Compiling the ANN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the ANN to the Training set
regressor.fit(X_train, y_train, batch_size = 32, epochs = 200)


############ Save & load Trained Model ############

# Save Trained Model
regressor.save('TICKER-BKP.h5')

# deletes the existing model
del regressor

# load Trained Model
regressor = load_model('TICKER-BKP.h5')

############ Predict & Test the Model ############
real_stock_price = np.array(X_test)
inputs = real_stock_price
predicted_stock_price = regressor.predict(inputs)

# rebuild the Structure
dataset_test_total = pd.DataFrame()
dataset_test_total['real'] = real_stock_price
dataset_test_total['predicted'] = predicted_stock_price
# real data price VS. predicted price
predicted_stock_price = scaler.inverse_transform(dataset_test_total) 

# count of Wrong predicted value after applying treshold
err_cnt = error_count(predicted_stock_price[:, 0], predicted_stock_price[:, 1], toler_treshold = 5.0)

# Calc difference between real data price and predicted price
diff_rate = calc_diff(predicted_stock_price[:, 0], predicted_stock_price[:, 1])

# MSE
mse = mean_squared_error(predicted_stock_price[:, 0], predicted_stock_price[:, 1])

############ Visualizing the results ############
inputs = np.array(X)

all_real_price = np.array(y)
all_predicted_price = regressor.predict(inputs)

# rebuild the Structure
dataset_pred_real = pd.DataFrame()
dataset_pred_real['real'] = all_real_price
dataset_pred_real['predicted'] = all_predicted_price

# real test data price VS. predicted price
all_prices = scaler.inverse_transform(dataset_pred_real)  

Epoch 1/200


2022-02-21 09:11:24.957095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:08:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-02-21 09:11:24.957329: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-21 09:11:24.957381: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-02-21 09:11:24.957416: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-02-21 09:11:24.957448: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

25/25 [==============================] - 0s 3ms/step - loss: 0.0510
Epoch 2/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0247
Epoch 3/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0112
Epoch 4/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0098
Epoch 5/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0097
Epoch 6/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0100
Epoch 7/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0097
Epoch 8/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0101
Epoch 9/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0101
Epoch 10/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0101
Epoch 11/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0096
Epoch 12/200
25/25 [==============================] - 0s 3ms/step - loss: 0.0096
Epoch 13/200
25/25 [=============================

NameError: name 'error_count' is not defined